In [25]:
import tensorflow as tf
import simplejson
import matplotlib.pyplot as plt
%matplotlib inline
import threading
import tensorflow.contrib.slim as slim

In [27]:
def arg_scope(H, phase):
    '''
    This returns the arg_scope for slim.arg_scope(), which defines the options for slim.functions
    '''
    padding = H['padding']
    is_training = {'train': True, 'validate': False}[phase]
    pool_kernel = [2, 2]
    pool_stride = 2

    
    params = {
        "decay": 0.997,
        "epsilon": 0.001,
    }

    with slim.arg_scope(
            func=slim.conv2d, 
            activation_fn=slim.relu, 
            padding=padding, 
            normalizer_fn=slim.batch_norm, 
            weights_initializer=tf.contrib.layers.variance_scaling_initializer()):
        with slim.arg_scope(func=[slim.batch_norm, slim.dropout], is_training=is_training):
            with slim.arg_scope(func=slim.max_pool2d, stride=pool_stride, kernel_size=pool_kernel) as sc:
                with slim.arg_scope(
                    func=slim.conv2d_transpose, 
                    stride=pool_stride, 
                    kernel_size=pool_kernel, 
                    padding=padding):
                    return sc

In [7]:
def build_pred(x_in, H, phase):
    '''
    This function builds the prediction model
    '''
    conv_kernel_1 = [1, 1]
    conv_kernel_3 = [3, 3]
    
    early_feature = {}
    
    with slim.arg_scope(arg_scope(H, phase)):
        
        scope_name = 'block_1'
        x_input = x_in
        num_outputs = 64
        with tf.variable_scope(values=scope_name):
            lay_1 = slim.conv2d(inputs=x_input, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            lay_2 = slim.conv2d(inputs=layer_1, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            early_feature[scope_name] = layer_2
        
        scope_name = 'block_2'
        x_input = slim.max_pool2d(inputs=layer_2)
        num_outputs = 128
        with tf.variable_scope(values=scope_name):
            lay_1 = slim.conv2d(inputs=x_input, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            lay_2 = slim.conv2d(inputs=layer_1, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            early_feature[scope_name] = layer_2

        scope_name = 'block_3'
        x_input = slim.max_pool2d(inputs=layer_2)
        num_outputs = 256
        with tf.variable_scope(values=scope_name):
            lay_1 = slim.conv2d(inputs=x_input, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            lay_2 = slim.conv2d(inputs=layer_1, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            early_feature[scope_name] = layer_2
            
        scope_name = 'block_4'
        x_input = slim.max_pool2d(inputs=layer_2)
        num_outputs = 512
        with tf.variable_scope(values=scope_name):
            lay_1 = slim.conv2d(inputs=x_input, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            lay_2 = slim.conv2d(inputs=layer_1, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            early_feature[scope_name] = layer_2

        scope_name = 'block_5'
        x_input = slim.max_pool2d(inputs=layer_2)
        num_outputs = 1024
        with tf.variable_scope(values=scope_name):
            lay_1 = slim.conv2d(inputs=x_input, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            lay_2 = slim.conv2d(inputs=layer_1, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            early_feature[scope_name] = layer_2
            
        scope_name = 'block_6'
        num_outputs = 512
        trans_layer_2 = slim.conv2d_transpose(inputs=slim.max_pool2d(layer_2), num_outputs=num_outputs)
        x_input = slim.max_pool2d(inputs=tf.concat([early_features['block_5'], trans_layer_2], axis=3))
        with tf.variable_scope(values=scope_name):
            lay_1 = slim.conv2d(inputs=x_input, kernel_size=conv_kernel_3, num_outputs=num_outputs)
            lay_2 = slim.conv2d(inputs=layer_1, kernel_size=conv_kernel_3, num_outputs=num_outputs)
        return layer_2

In [9]:
def build_loss(x_in, H, phase, phase):
    '''
    This function builds the loss and accuracy
    '''
    reuse = {'train': False, 'validate': True}[phase]
    with tf.variable_scope(reuse=reuse=se):
        build_pred(x_in, H)

In [8]:
def build(queues, H):
    '''
    This function returns the train operation, summary, global step
    '''
    im_width = H['im_width']
    im_height = H['im_height']
    num_class = H['num_class']
    num_channel = H['num_channel']
    batch_size = H['batch_size']
    loss, accuracy = {}, {}
    for phase in ['train', 'validate']:
        x_in = queues[phase].dequeue_many(batch_size)
        loss[phase], accuracy[phase] = build_loss(x_in, H, phase)
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = 
    

In [26]:
hypes = './hypes/hypes.json'
with open(hypes, 'r') as f:
    H = simplejson.load(f)
    im_width = H['im_width']
    im_height = H['im_height']
    num_class = H['num_class']
    num_channel = H['num_channel']
    queue_size = H['queue_size']

In [12]:
def enqueue_thread(sess, data_gen, coord, phase, enqueue_op):
    while not coord.should_stop():
        sess.run(enqueue_op, feed_dict={x_in[phase]: data_gen.next()})

In [21]:
x_in, queues, enqueue_op = {}, {}, {}
shape = ((im_width, im_height, num_channel), 
         (im_width, im_height, num_class)
    )
for phase in ['train', 'validate']:
    x_in[phase] = tf.placeholder(dtype=tf.float32)
    queues[phase] = tf.FIFOQueue(capacity=queue_size, shapes=shape, dtypes=(tf.float32,tf.float32))
    enqueue_op[phase] = queues[phase].enqueue(x_in[phase])

In [ ]:
build(queues, H)

In [23]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
coord = tf.train.Coordinator()
threads = {}
with tf.Session(config=config) as sess:
    for phase in ['train', 'valicate']:
        threads[phase] = threading.Thread(
            target=enqueue_thread, 
            args=(sess, data_gen[phase], coord, phase, enqueue_op[phase]))
        threads[phase].start()

KeyError: 'valicate'

In [ ]:
coord.requst_stop()
coord.join()